In [1]:
import json
from functools import reduce
from joblib import Parallel, delayed
from pathlib import Path

### Домашняя работа

* Сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм.
* Реализация через цикл (предпогаем, что мы не знаем сколько у нас фильмов):

In [2]:
n, mean, M2 = 0, 0.0, 0
for path in Path('data/archive/').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774862


In [3]:
def mapper(path):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
    
        return (float(info['movieIMDbRating']),)
    

def reducer(score_data1, score_data2):
    if score_data1 is None and score_data2 is None:
        return None
    
    elif score_data1 is None:
        return score_data2
    
    elif score_data2 is None:
        return score_data1
    
    else:
        scores = []
        if len(score_data1) == 1.0:
            n, mean, M2 = 0, 0.0, 0
            scores.append(score_data1[0])
        else:
            n, mean, M2 = score_data1
        scores.append(score_data2[0])
        
        for score in scores:
            n += 1
            delta = score - mean
            mean += delta / n
            M2 += delta * (score - mean)
            
        return n, mean, M2

In [4]:
%%time

n, mean, M2 = reduce(reducer, map(mapper, Path('data/archive/').glob('**/*')))
print(mean, (M2 / n) ** (1/2))

8.03 1.0517128885774862
CPU times: user 3.04 ms, sys: 1.84 ms, total: 4.88 ms
Wall time: 3.6 ms


In [5]:
%%time

paths = list(Path('data/archive/').glob('**/*')) * 10_000

n, mean, M2 = reduce(reducer, Parallel(n_jobs=16)(delayed(mapper)(path) for path in paths))
print(mean, (M2 / n) ** (1/2))

8.030000000000113 1.0517128885774867
CPU times: user 3.16 s, sys: 234 ms, total: 3.4 s
Wall time: 3.74 s
